In [ ]:
import numpy as np  # Импорт библиотеки NumPy

class RNN:  # Определение класса RNN
    def __init__(self, learning_rate, epochs, hidden_size):  # Инициализация параметров класса
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.hidden_size = hidden_size
        self.Whh = np.random.randn(hidden_size, hidden_size)  # Инициализация весов скрытого слоя
        self.Wxh = np.random.randn(hidden_size, 1)  # Инициализация весов входного слоя
        self.bh = np.zeros((hidden_size, 1))  # Инициализация смещений скрытого слоя
        self.mean = None  # Переменная для хранения среднего значения
        self.std = None  # Переменная для хранения стандартного отклонения

    def normalize(self, data):  # Метод для нормализации данных
        self.mean = np.mean(data)  # Вычисление среднего значения
        self.std = np.std(data)  # Вычисление стандартного отклонения
        return (data - self.mean) / self.std  # Нормализация данных

    def denormalize(self, data):  # Метод для обратной нормализации данных
        return data * self.std + self.mean  # Обратная нормализация данных

    def train(self, bitcoin_prices):  # Метод для обучения модели
        bitcoin_prices_normalized = self.normalize(bitcoin_prices)  # Нормализация входных данных
        for epoch in range(self.epochs):  # Цикл обучения по эпохам
            loss = 0  # Инициализация функции потерь
            h_prev = 0  # Инициализация предыдущего скрытого состояния

            for i in range(len(bitcoin_prices_normalized) - 1):  # Цикл обучения по временным точкам
                # Прямой проход
                h = np.tanh(np.dot(self.Whh, h_prev) + np.dot(self.Wxh, bitcoin_prices_normalized[i]) + self.bh)

                # Предсказание
                predicted_price = np.dot(self.Wxh, bitcoin_prices_normalized[i + 1])

                # Вычисление ошибки
                loss += (predicted_price - bitcoin_prices_normalized[i + 1]) ** 2

                # Обратный проход
                dWxh = np.dot((predicted_price - bitcoin_prices_normalized[i + 1]), bitcoin_prices_normalized[i + 1])
                dWhh = np.dot((predicted_price - bitcoin_prices_normalized[i + 1]), h_prev)
                dbh = (predicted_price - bitcoin_prices_normalized[i + 1])

                # Обновление весов
                self.Wxh -= self.learning_rate * dWxh
                self.Whh -= self.learning_rate * dWhh
                self.bh -= self.learning_rate * dbh

                h_prev = h  # Обновление предыдущего скрытого состояния

            # Вывод ошибки на каждой эпохе
            print(f'Epoch {epoch + 1}, Loss: {loss}')

    def predict(self, bitcoin_prices):  # Метод для прогнозирования
        bitcoin_prices_normalized = self.normalize(bitcoin_prices)  # Нормализация входных данных
        h_prev = 0  # Инициализация предыдущего скрытого состояния
        for i in range(len(bitcoin_prices_normalized) - 1):  # Цикл для прогнозирования
            h = np.tanh(np.dot(self.Whh, h_prev) + np.dot(self.Wxh, bitcoin_prices_normalized[i]) + self.bh)
            h_prev = h  # Обновление предыдущего скрытого состояния

        # Прогнозирование будущего значения
        next_price_normalized = np.tanh(np.dot(self.Whh, h) + np.dot(self.Wxh, bitcoin_prices_normalized[-1]) + self.bh)
        next_price = self.denormalize(next_price_normalized)  # Обратная нормализация прогноза
        return next_price  # Возврат прогноза

# Создание экземпляра класса RNN и обучение модели
rnn = RNN(learning_rate=0.01, epochs=50, hidden_size=1)  # Создание экземпляра класса RNN
bitcoin_prices = np.array([100, 108, 110, 115, 105, 112, 120, 125, 135, 140])  # Исторические данные курса биткоина
rnn.train(bitcoin_prices)  # Обучение модели на исторических данных

# Прогнозирование следующего значения
next_price = rnn.predict([140, 145, 155, 160])  # Прогноз следующего значения курса биткоина
print(f'Predicted next Bitcoin price: {next_price}')  # Вывод прогноза
